# Ordinary Differential Equations (ODEs)

- [DifferentialEquations.jl docs](https://diffeq.sciml.ai/dev/index.html)

## General steps to solve ODEs (in the old way)

- Define a model function representing the right-hand-side (RHS) of the sysstem.
  - Out-of-place form: `f(u, p, t)` where `u` is the state variable(s), `p` is the parameter(s), and `t` is the independent variable (usually time). The output is the right hand side (RHS) of the differential equation system.
  - In-place form: `f!(du, u, p, t)`, where the output is saved to `du`. The rest is the same as the out-of-place form. The in-place function may be faster since it does not allocate a new array in each call.
- Initial conditions (`u0`) for the state variable(s).
- (Optional) parameter(s) `p`.
- Define a problem (e.g. `ODEProblem`) using the model function (`f`), initial condition(s) (`u0`), simulation time span (`tspan == (tstart, tend)`), and parameter(s) `p`.
- Solve the problem by calling `solve(prob)`.

### Radioactive decay

The decaying rate of a nuclear isotope is proprotional to its concentration (number):

$$
\frac{d}{dt}C(t) = - \lambda C(t)
$$

**State variable(s)**

- $C(t)$: The concentration (number) of a decaying nuclear isotope.

**Parameter(s)**

- $\lambda$: The rate constant of nuclear decay. The half-life: $t_{\frac{1}{2}} = \frac{ln2}{\lambda}$.

In [ ]:
using DifferentialEquations
using Plots
Plots.default(fmt=:png)

The exponential decay ODE model, out-of-place (3-parameter) form

In [ ]:
expdecay(u, p, t) = p * u

In [ ]:
p = -1.0 ## Parameter
u0 = 1.0 ## Initial condition
tspan = (0.0, 2.0) ## Simulation start and end time points
prob = ODEProblem(expdecay, u0, tspan, p) ## Define the problem
sol = solve(prob) ## Solve the problem

Visualize the solution with `Plots.jl`.

In [ ]:
plot(sol, label="Exp decay")

Solution handling: https://docs.sciml.ai/DiffEqDocs/stable/basics/solution/

The mostly used feature is `sol(t)`, the state variables at time `t`. `t` could be a scalar or a vector-like sequence. The result state variables are calculated with interpolation. 

In [ ]:
sol(1.0)

In [ ]:
sol(0.0:0.1:2.0)

`sol.t`: time points by the solver.

In [ ]:
sol.t

`sol.u`: state variables at `sol.t`.

In [ ]:
sol.u

### The SIR model

A more complicated example is the [SIR model](https://www.maa.org/press/periodicals/loci/joma/the-sir-model-for-spread-of-disease-the-differential-equation-model) describing infectious disease spreading. There are 3 state variables and 2 parameters.

$$
\begin{align}
\frac{d}{dt}S(t) &= - \beta S(t)I(t)  \\
\frac{d}{dt}I(t) &= \beta S(t)I(t)  - \gamma I(t)  \\
\frac{d}{dt}R(t) &= \gamma I(t)
\end{align}
$$

**State variable(s)**

- $S(t)$ : the fraction of susceptible people
- $I(t)$ : the fraction of infectious people
- $R(t)$ : the fraction of recovered (or removed) people

**Parameter(s)**

- $\beta$ : the rate of infection when susceptible and infectious people meet
- $\gamma$ : the rate of recovery of infectious people

In [ ]:
using DifferentialEquations
using Plots

Here we use the in-place form: `f!(du, u, p ,t)` for the SIR model. The output is written to the first argument `du`, without allocating a new array in each function call.

In [ ]:
function sir!(du, u, p, t)
	s, i, r = u
	β, γ = p
	v1 = β * s * i
	v2 = γ * i
    du[1] = -v1
    du[2] = v1 - v2
    du[3] = v2
	return nothing
end

Setup parameters, initial conditions, time span, and the ODE problem.

In [ ]:
p = (β = 1.0, γ = 0.3)
u0 = [0.99, 0.01, 0.00]
tspan = (0.0, 20.0)
prob = ODEProblem(sir!, u0, tspan, p)

Solve the problem

In [ ]:
sol = solve(prob)

Visualize the solution

In [ ]:
plot(sol, labels=["S" "I" "R"], legend=:right)

`sol[i]`: all components at timestep `i`

In [ ]:
sol[2]

`sol[i, j]`: `i`th component at timestep `j`

In [ ]:
sol[1, 2]

`sol[i, :]`: the timeseries for the `i`th component.

In [ ]:
sol[1, :]

`sol(t,idxs=1)`: the 1st element in time point(s) `t` with interpolation. `t` can be a scalar or a vector.

In [ ]:
sol(10, idxs=2)

In [ ]:
sol(0.0:0.1:20.0, idxs=2)

### Lorenz system

The Lorenz system is a system of ordinary differential equations having chaotic solutions for certain parameter values and initial conditions. ([Wikipedia](https://en.wikipedia.org/wiki/Lorenz_system))

$$
\begin{align}
  \frac{dx}{dt} &=& \sigma(y-x) \\
  \frac{dy}{dt} &=& x(\rho - z) -y \\
  \frac{dz}{dt} &=& xy - \beta z
\end{align}
$$

In this example, we will use NamedTuple for the parameters and [ComponentArrays.jl](https://github.com/jonniedie/ComponentArrays.jl) for the state variaBLE to access elements by their names.

In [ ]:
using ComponentArrays
using DifferentialEquations
using Plots

In [ ]:
function lorenz!(du, u, p, t)
    du.x = p.σ*(u.y-u.x)
    du.y = u.x*(p.ρ-u.z) - u.y
    du.z = u.x*u.y - p.β*u.z
    return nothing
end

In [ ]:
u0 = ComponentArray(x=1.0, y=0.0, z=0.0)
p = (σ=10.0, ρ=28.0, β=8/3)
tspan = (0.0, 100.0)
prob = ODEProblem(lorenz!, u0, tspan, p)
sol = solve(prob)

`idxs=(1, 2, 3)` makes a phase plot with 1st, 2nd, and the 3rd state variable.

In [ ]:
plot(sol, idxs=(1, 2, 3), size=(400,400))

The plot recipe is using interpolation for smoothing. You can turn off `denseplot` to see the difference.

In [ ]:
plot(sol, idxs=(1, 2, 3), denseplot=false, size=(400,400))

The zeroth variable in `idxs` is the independent variable (usually time). The FOLLOWING command plots the time series of the second state variable (`y`).

In [ ]:
plot(sol, idxs=(0, 2))

### Non-autonomous ODEs

In non-autonomous ODEs, term(s) in the right-hadn-side (RHS) maybe time-dependent. For example, in this pendulum model has an external, time-dependent, force.

$$
\begin{aligned}
\dot{\theta} &= \omega(t) \\
\dot{\omega} &= -1.5\frac{g}{l}sin(\theta(t)) + \frac{3}{ml^2}M(t)
\end{aligned}
$$

- $\theta$: pendulum angle
- $\omega$: angular rate
- M: time-dependent external torgue
- $l$: pendulum length
- $g$: gravitional acceleration

In [ ]:
using DifferentialEquations
using Plots

In [ ]:
function pendulum!(du, u, p, t)
    l = 1.0                             ## length [m]
    m = 1.0                             ## mass [kg]
    g = 9.81                            ## gravitational acceleration [m/s²]
    du[1] = u[2]                        ## θ'(t) = ω(t)
    du[2] = -3g/(2l)*sin(u[1]) + 3/(m*l^2)*p(t) # ω'(t) = -3g/(2l) sin θ(t) + 3/(ml^2)M(t)
end

u0 = [0.01, 0.0]                     ## initial angular deflection [rad] and angular velocity [rad/s]
tspan = (0.0, 10.0)                  ## time interval

M = t -> 0.1 * sin(t)                   ## external torque [Nm] as the parameter for the pendulum model

prob = ODEProblem(pendulum!, u0, tspan, M)
sol = solve(prob)

plot(sol, linewidth=2, xaxis="t", label=["θ [rad]" "ω [rad/s]"])
plot!(M, tspan..., label="Ext. force")

### Linear ODE system

The ODE system could be anything as long as it returns the derivatives of state variables. In this example, the ODE system is described by a matrix differential operator.

$\dot{u} = Au$

In [ ]:
using DifferentialEquations
using Plots

A = [
    1. 0  0 -5
    4 -2  4 -3
    -4  0  0  1
    5 -2  2  3
]

u0 = rand(4, 2)

tspan = (0.0, 1.0)
f = (u, p, t) -> A*u
prob = ODEProblem(f, u0, tspan)
sol = solve(prob)
plot(sol)

## ModelingToolkit: define ODEs with symbolic expressions

You can also define ODE systems symbolically using [ModelingToolkit.jl (MTK)](https://github.com/SciML/ModelingToolkit.jl) and let MTK generate high-performace ODE functions.

- transforming and simplifying expressions for performance
- Jacobian and Hessian generation
- accessing eliminated states (called observed variables)
- building and connecting subsystems programatically (component-based modeling)

See also [Simulating Big Models in Julia with ModelingToolkit @ JuliaCon 2021 Workshop](https://youtu.be/HEVOgSLBzWA).

### Radioactive decay

Here we use the same example of radioactive decay

In [ ]:
using ModelingToolkit
using DifferentialEquations
using Plots

In [ ]:
## independent variable (time) and dependent variables
@variables t c(t) RHS(t)

## parameters: decay rate
@parameters λ

## Differential operator w.r.t. time
D = Differential(t)

Equations in MTK use the tilde character (`~`) for equality.

Every MTK system requires a name. The `@named` macro simply ensures that the symbolic name matches the name in the REPL.

In [ ]:
eqs = [
    RHS ~ -λ * c
    D(c) ~ RHS
]

@named sys = ODESystem(eqs, t)

`structural_simplify()` simplifies the two equations to one.

In [ ]:
sys = structural_simplify(sys)

Setup initial conditions, time span, parameter values, the `ODEProblem`, and solve the problem.

In [ ]:
p = [λ => 1.0]
u0 = [c => 1.0]
tspan = (0.0, 2.0)
prob = ODEProblem(sys, u0, tspan, p)
sol = solve(prob)

Visualize the solution with `Plots.jl`.

In [ ]:
plot(sol, label="Exp decay")

The solution interface provides symbolic access. So you can access the results of `c` directly.

In [ ]:
sol[c]

In [ ]:
sol(0.0:0.1:2.0, idxs=c)

The eliminated term (RHS in this example) is still tracible.

In [ ]:
plot(sol, idxs=[c, RHS], legend=:right)

The symbolic interface allows calculation of states.

In [ ]:
plot(sol, idxs=[c*1000])

### Lorenz system

We use the same Lorenz system example as above. Here we setup the initial conditions and parameters with default values.

In [ ]:
@variables t x(t)=1.0 y(t)=0.0 z(t)=0.0
@parameters (σ=10.0, ρ=28.0, β=8/3)

D = Differential(t)

eqs = [
    D(x) ~ σ * (y - x)
    D(y) ~ x * (ρ - z) - y
    D(z) ~ x * y - β * z
]

@named sys = ODESystem(eqs, t)

Here we are using default values, so we pass empty arrays for initial conditions and parameter values.

In [ ]:
sys = structural_simplify(sys)
tspan = (0.0, 100.0)
prob = ODEProblem(sys, [], tspan, [])
sol = solve(prob)

Plot the solution with symbols instead of index numbers.

In [ ]:
plot(sol, idxs=(x, y, z), size=(400,400))

### Non-autonomous ODEs

Sometimes a model might have a time-variant external force, which is too complex or impossible to express it symbolically. In such situation, one could apply `@register_symbolic` to it to exclude it from symbolic transformations and use it numberically.

In [ ]:
@variables t x(t) f(t)
@parameters τ
D = Differential(t)

Define a time-dependent random external force

In [ ]:
value_vector = randn(10)

f_fun(t) = t >= 10 ? value_vector[end] : value_vector[Int(floor(t))+1]

"Register" arbitrary Julia functions to be excluded from symbolic transformations. Just use it as-is (numberically).

In [ ]:
@register_symbolic f_fun(t)

In [ ]:
@named fol_external_f = ODESystem([f ~ f_fun(t), D(x) ~ (f - x)/τ], t)

In [ ]:
sys = structural_simplify(fol_external_f)
prob = ODEProblem(sys, [x => 0.0], (0.0, 10.0), [τ => 0.75])
sol = solve(prob)
plot(sol, idxs=[x, f])

### Second-order ODE systems

`ode_order_lowering(sys)` automatically transforms a second-order ODE into two first-order ODEs. 

In [ ]:
using Plots
using ModelingToolkit
using DifferentialEquations

@parameters σ ρ β
@variables t x(t) y(t) z(t)
D = Differential(t)

In [ ]:
eqs = [
    D(D(x)) ~ σ * (y-x),
    D(y) ~ x * (ρ - z) - y,
    D(z) ~ x * y - β * z
]

In [ ]:
@named sys = ODESystem(eqs, t)
sys = sys |> ode_order_lowering |> structural_simplify

Note that one needs to provide the initial condition for the first derivative.

In [ ]:
u0 = [
    D(x) => 2.0,
    x => 1.0,
    y => 0.0,
    z => 0.0
]

p = [
    σ => 28.0,
    ρ => 10.0,
    β => 8/3
]

tspan = (0.0, 100.0)
prob = ODEProblem(sys, u0, tspan, p, jac=true)
sol = solve(prob)
plot(sol, idxs=(x, y, z), label="Trajectory", size=(500,500))

### Composing systems

By defining connection equation(s) to couple ODE systems together, we can build component-based, hierarchical models.

In [ ]:
using Plots
using ModelingToolkit
using DifferentialEquations

@parameters σ ρ β
@variables t x(t) y(t) z(t)

D = Differential(t)

eqs = [
    D(x) ~ σ * (y - x),
    D(y) ~ x * (ρ - z) - y,
    D(z) ~ x * y - β * z
]

@named lorenz1 = ODESystem(eqs)
@named lorenz2 = ODESystem(eqs)

Define relations (connectors) between the two systems.

In [ ]:
@variables a(t)
@parameters γ

connections = [0 ~ lorenz1.x + lorenz2.y + a * γ]

@named connLorenz = ODESystem(connections, t , [a], [γ], systems = [lorenz1, lorenz2])

All state variables in the combined system

In [ ]:
states(connLorenz)

In [ ]:
u0 = [
    lorenz1.x => 1.0, lorenz1.y => 0.0, lorenz1.z => 0.0,
    lorenz2.x => 0.0, lorenz2.y => 1.0, lorenz2.z => 0.0,
    a => 2.0
]

p = [
    lorenz1.σ => 10.0, lorenz1.ρ => 28.0, lorenz1.β => 8/3,
    lorenz2.σ => 10.0, lorenz2.ρ => 28.0, lorenz2.β => 8/3,
    γ => 2.0
]

tspan = (0.0, 100.0)
sys = connLorenz |> structural_simplify
sol = solve(ODEProblem(sys, u0, tspan, p, jac=true))
plot(sol, idxs=(a, lorenz1.x, lorenz2.x), size=(600,600))

### Convert existing functions into MTK systems

`modelingtoolkitize(prob)` generates MKT systems from regular DE problems. I t can also generate analytic Jacobin functions for faster solving.

Example: **[DAE index reduction](https://mtk.sciml.ai/stable/mtkitize_tutorials/modelingtoolkitize_index_reduction/)** for the pendulum problem, which cannot be solved by regular ODE solvers.

In [ ]:
using Plots
using ModelingToolkit
using DifferentialEquations
using LinearAlgebra

function pendulum!(du, u, p, t)
    x, dx, y, dy, T = u
    g, L = p
    du[1] = dx
    du[2] = T*x
    du[3] = dy
    du[4] = T*y - g
    # Do not write your function like this after you've learned MTK
    du[5] = x^2 + y^2 - L^2
    return nothing
end

In [ ]:
pendulum_fun! = ODEFunction(pendulum!, mass_matrix = Diagonal([1, 1, 1, 1, 0]))
u0 = [1.0, 0.0, 0.0, 0.0, 0.0]
p = [9.8, 1.0]
tspan = (0.0, 10.0)
pendulum_prob = ODEProblem(pendulum_fun!, u0, tspan, p)

Convert the ODE problem to a MTK system.

In [ ]:
tracedSys = modelingtoolkitize(pendulum_prob)

`structural_simplify()` and `dae_index_lowering()` transform the index-3 DAE into an index-0 ODE.

In [ ]:
pendulumSys = tracedSys |> dae_index_lowering |> structural_simplify

The default `u0` is included in the system already so one can use an empty array `[]` as the initial conditions.

In [ ]:
prob = ODAEProblem(pendulumSys, [], tspan)
sol = solve(prob, abstol=1e-8, reltol=1e-8)
plot(sol, idxs=states(tracedSys))